In [0]:
from google.colab import files

In [0]:
files.upload()

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d oddrationale/mnist-in-csv

In [0]:
!unzip mnist-in-csv.zip

In [0]:
!ls

In [0]:
!head mnist_train.csv

In [0]:
import pandas as pd
import tensorflow as tf

In [0]:
TRAIN_FILE_PATH = 'mnist_train.csv'
TEST_FILE_PATH  = 'mnist_test.csv'

In [0]:
LABEL_COLUMN = 'label'

In [0]:
train_df = pd.read_csv(TRAIN_FILE_PATH)
test_df  = pd.read_csv(TEST_FILE_PATH)
target_train = pd.get_dummies(train_df.pop('label'))
target_test  = pd.get_dummies(test_df.pop('label'))

In [0]:
train_df

In [0]:
train_df

In [0]:
train_df = train_df.values.reshape(60000, 28, 28, 1)
test_df  = test_df.values.reshape(10000, 28, 28, 1)

In [0]:
target_train = target_train.values
target_test  = target_test.values

In [0]:
target_train

In [0]:
dataset_train = tf.data.Dataset.from_tensor_slices((train_df, target_train)).shuffle(126).batch(32)
dataset_test  = tf.data.Dataset.from_tensor_slices((test_df, target_test)).shuffle(126).batch(32)

In [0]:
import matplotlib.pyplot as plt
import numpy as np

In [0]:
def visualisze_Dataset(dataset):
  fig = plt.figure(figsize=(10, 10))
  i = 0
  for data_ in dataset.take(1):
    features, labels =  data_
    for i in range(1, 25+1):
      fig.add_subplot(5, 5, i)
      plt.imshow(features[i-1].numpy().reshape(28, 28))
      plt.title(np.argmax(labels[i-1].numpy()))
      i+=1
      # print(d[0].numpy())
    break

In [0]:
visualisze_Dataset(dataset_test)

In [0]:
from tensorflow.keras import Sequential, layers, losses, optimizers

In [0]:
model = Sequential([
                    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
                    layers.Conv2D(64, (3, 3), activation='relu'),
                    layers.MaxPool2D((2,2)),
                    layers.Flatten(),
                    layers.Dense(728, activation='relu'),
                    layers.Dense(512, activation='relu'),
                    layers.Dense(128, activation='relu'),
                    layers.Dense(10, activation='softmax')
])

In [0]:
model.compile(loss=losses.CategoricalCrossentropy(),metrics=['accuracy'], optimizer=optimizers.Adam(0.001))

In [0]:
model.fit(dataset_train,epochs=10,validation_data=dataset_test)